In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import interpolate


width = 30 # snakemake.config["width"]
height = 60 # snakemake.config["height"]
scale = 1

In [3]:
def draw_trajectory(df, ax):
    x = df.path_x / scale
    y = df.path_y / scale
    ax.plot(x, y, alpha=0.75)

In [4]:
def _df_interpolate(df, max_=5):
    def per_df(d):
        x = np.array(d['path_x'])
        y = np.array(d['path_y'])

        # remove two consecutive equal points for interpolation
        okay = np.where(np.abs(np.diff(x)) + np.abs(np.diff(y)) > 0)
        x = np.r_[x[okay], x[-1]]
        y = np.r_[y[okay], y[-1]]

        tck,u=interpolate.splprep([x,y],s=0.0)
        x_i,y_i= interpolate.splev(np.linspace(0,1,(len(x)*10)),tck)

        new_rows = d.iloc[-1].to_dict()
        new_rows.update({'path_x' : x_i, 'path_y': y_i})
        return pd.DataFrame(new_rows)
    dfs = []
    agents = set(df["ant_nb"])
    trials = set(df["trial_nb"])
    if len(trials) > max_:
        trials = [arr[0] for arr in np.array_split(list(trials), max_)]
    #print(np.array_split(list(trials), max_))
    for agent in agents:
        for trial in trials:
            #print("tr", trial)
            #print(df[(df["ant_nb"]==agent) & (df["trial_nb"]==trial)])
            dfs.append(per_df(df[(df["ant_nb"]==agent) & (df["trial_nb"]==trial)]))
    return pd.concat(dfs, ignore_index=True)


In [5]:
df = pd.read_csv("trap_interpolated.csv")
df_raw = pd.read_csv("trap.csv")

In [8]:
fig, ax = plt.subplots(1, 1, figsize=(10 ,10))

#example_run = df[(df.ant_nb == 32) & (df.trial_nb == 1)]
for ant in [32]:#pd.unique(df["ant_nb"]):
    ant_df = df[df.ant_nb == ant]

    for run in pd.unique(ant_df["trial_nb"]):

        traj = ant_df[ant_df.trial_nb == run]
        draw_trajectory(traj, ax)


In [9]:
fig, ax = plt.subplots(1, 1, figsize=(10 ,10))

#example_run = df[(df.ant_nb == 32) & (df.trial_nb == 1)]
for ant in [32]:#pd.unique(df["ant_nb"]):
    ant_df = df[df.ant_nb == ant]

    for run in pd.unique(ant_df["trial_nb"]):

        traj = ant_df[ant_df.trial_nb == run]
        draw_trajectory(_df_interpolate(traj), ax)

In [37]:

for ant in [32]:#pd.unique(df["ant_nb"]):
    ant_df = df[df.ant_nb == ant]

    for run in pd.unique(ant_df["trial_nb"]):
        fig, ax = plt.subplots(1, 1, figsize=(10 ,10))

        traj = ant_df[ant_df.trial_nb == run]
        traj.plot(x="path_x", y="path_y", ax=ax)        
        traj.rolling(10).mean().plot(x="path_x", y="path_y", ax=ax)

# make heatmap?
